# Deep Learning with TensorFlow
### Week 4: Normalising flows

## Contents

[1. Introduction](#introduction)

[2. Change of variables formula](#changeofvariables)

[3. Distributions (\*)](#distributions)

[4. Bijectors (\*)](#bijectors)

[5. NICE / RealNVP](#nicerealnvp)

[6. Bijector subclassing (\*)](#bijector_subclassing)

[References](#references)

<a class="anchor" id="introduction"></a>
## Introduction

So far in this module we have covered many of the fundamental building blocks of deep learning: from mathematical neurons and multilayer perceptrons, to optimisation and regularisation of deep learning models, and the important CNN architecture.

In the remaining two weeks of the module, we will use these building blocks to focus our attention on the probabilistic approach to deep learning. This is an increasingly popular branch of deep learning that aims to make use of tools from probability theory to account for noise and uncertainty in the data. Probabilistic deep learning models make direct use of probability distributions and latent random variables in the model architecture.

In this week of the course we will begin to familiarise ourselves with the [TensorFlow Probability](https://www.tensorflow.org/probability/) (TFP) library, which is built on TensorFlow to enable a closer integration between deep learning, probabilistic modelling and statistical analysis. In particular, we will learn about `Distribution` and `Bijector` objects in TFP.

These objects will provide the tools we need to develop normalising flow deep learning models. Normalising flows are a class of generative models, that were first popularised in the context of variational inference by [Rezende & Mohamed 2015](#Rezende15), and in the context of density estimation by [Dinh et al 2015](#Dinh15). In this week, we will focus on using normalising flows to estimate continuous data distributions.

When trained as a density estimator, this type of model is able to produce new instances that could plausibly have come from the same dataset that it is trained on, as well as tell you whether a given example instance is likely. However, for complex datasets the data distribution can be very difficult to model, so this is a highly nontrivial task in general. This is where the power of deep learning models can be leveraged to learn highly multimodal and complicated data distributions, and this type of model has been successfully applied to domains such as image generation ([Ho et al 2019](#Ho19)), noise modelling ([Abdelhamed et al 2019](#Abdelhamed19)), audio synthesis ([Prenger et al 2019](#Prenger19)), and video generation ([Kumar et al 2019](#Kumar19)).

In this week, we will see how to construct the popular normalising flow architecture called RealNVP ([Dinh et al 2017](#Dinh17)).

<a class="anchor" id="changeofvariables"></a>
## Change of variables formula

The approach taken by normalising flows to solve the density estimation task is to take an initial, simple density, and transform it - possibly using a series of parameterised transformations - to produce a rich and complex distribution. 

If these transformations are smooth and invertible, then we are able to evaluate the density of the complex transformed distribution. This property is important, because it then allows to train such a model using maximum likelihood. This is the idea behind normalising flows. The invertible transformations themselves are what constitute the bijectors module in the Tensorflow Probability library.

We'll start this week by reviewing the change of variables formula, which forms the mathematical basis of normalising flows.

#### Statement of the formula
Let $Z := (z_1,\ldots,z_D)\in\mathbb{R}^D$ be a $D$-dimensional continuous random variable, and suppose that $f:\mathbb{R}^D\mapsto\mathbb{R}^D$ is a smooth, invertible transformation. Now consider the change of variables $X = f(Z)$, with $X=(x_1,\ldots,x_D)$, and denote the probability density functions of the random variables $Z$ and $X$ by $p_Z$ and $p_X$ respectively.

The change of variables formula states that

$$
p_X(\mathbf{x}) = p_Z(\mathbf{z})\cdot\left|\det J_f(\mathbf{z}) \right|^{-1},\label{cov_f}\tag{1}
$$

where $\mathbf{x}, \mathbf{z}\in\mathbb{R}^D$, and $J_f(\mathbf{z})\in\mathbb{R}^{D\times D}$ is the **Jacobian** of the transformation $f$, given by the matrix of partial derivatives

$$
J_f(\mathbf{z}) = \left[ 
\begin{array}{ccc}
\frac{\partial f_1}{\partial z_1}(\mathbf{z}) & \cdots & \frac{\partial f_1}{\partial z_D}(\mathbf{z})\\
\vdots & \ddots & \vdots\\
\frac{\partial f_D}{\partial z_1}(\mathbf{z}) & \cdots & \frac{\partial f_D}{\partial z_d}(\mathbf{z})\\
\end{array}
\right],
$$

and $\left|\det J_f(\mathbf{z}) \right|$ is the absolute value of the determinant of the Jacobian matrix. Note that (1) can also be written in the log-form

$$
\log p_X(\mathbf{x}) = \log p_Z(\mathbf{z}) - \log \hspace{0.1ex}\left|\det J_f(\mathbf{z}) \right|. \label{logcov_f}\tag{2}
$$

Furthermore, we can equivalently consider the transformation $Z = f^{-1}(X)$. Then the change of variables formulae can be written as

$$
\begin{align}
p_Z(\mathbf{z}) &= p_X(\mathbf{x})\cdot\left|\det J_{f^{-1}}(\mathbf{x}) \right|^{-1},\label{cov_finv}\tag{3}\\
\log p_Z(\mathbf{z}) &= \log p_X(\mathbf{x}) - \log \hspace{0.1ex}\left|\det J_{f^{-1}}(\mathbf{x}) \right|.\label{logcov_finv}\tag{4}
\end{align}
$$

#### A simple example
We will demonstrate the change of variables formula with a simple example. Let $Z=(z_1, z_2)$ be a 2-dimensional random variable that is uniformly distributed on the unit square $[0, 1]^2 =: \Omega_Z$. We also define the transformation $f:\mathbb{R}^2 \mapsto \mathbb{R}^2$ as

$$
\begin{align}
f(z_1, z_2) = (\lambda z_1, \mu z_2)
\end{align}
$$

for some nonzero $\lambda, \mu\in\mathbb{R}$. The random variable $X=(x_1, x_2)$ is given by $X = f(Z)$. 

<img src="figures/change_of_variables.pdf" alt="Change of variables example in 2D" style="width: 750px;"/>
<center>Linearly transformed uniformly distributed random variable</center>

Since $\int_{\Omega_Z}p_Z(\mathbf{z})d\mathbf{z} = 1$ and $\mathbf{z}$ is uniformly distributed, we have that 

$$
p_Z(\mathbf{z}) = 1 \quad\text{for}\quad \mathbf{z}\in\Omega_Z.
$$

The random variable $X$ is uniformly distributed on the region $\Omega_X = f(\Omega_Z)$ as shown in the figure above (for the case $\lambda, \mu>0$). Since again $\int_{\Omega_X}p_X(\mathbf{x})d\mathbf{x} = 1$, the probability density function for $X$ must be given by 

$$
p_X(\mathbf{x}) = \frac{1}{|\Omega_X|} = \frac{1}{|\lambda\mu |}\quad\text{for}\quad \mathbf{x}\in\Omega_X.
$$

This result corresponds to the equations \eqref{cov_f}-\eqref{logcov_finv} above. In this simple example, the transformation $f$ is linear, and the Jacobian matrix is given by

$$
\begin{align}
J_f(\mathbf{z}) = \left[
\begin{array}{cc}
\lambda & 0\\
0 & \mu
\end{array}
\right].
\end{align}
$$

The absolute value of the determinant is $\left|\det J_{f^{-1}}(\mathbf{x}) \right| = |\lambda\mu | \ne 0$. Equation \eqref{cov_f} then implies

$$
\begin{align}
p_X(\mathbf{x}) &= p_Z(\mathbf{z})\cdot\left|\det J_f(\mathbf{z}) \right|^{-1}\\
&= \frac{1}{|\lambda\mu|}.
\end{align}
$$

Writing in the log-form as in equation \eqref{logcov_f} gives

$$
\begin{align}
\log p_X(\mathbf{x}) &= \log p_Z(\mathbf{z}) - \log \hspace{0.1ex}\left|\det J_f(\mathbf{z}) \right|\\
&= \log (1) - \log |\lambda\mu|\\
&= - \log |\lambda\mu|.
\end{align}
$$

#### Sketch of proof in 1-D
We now provide a sketch of the proof of the change of variables formula in one dimension. Let $Z$ and $X$ be random variables such that $X = f(Z)$, where $f : \mathbb{R}\mapsto\mathbb{R}$ is a $C^k$ diffeomorphism with $k\ge 1$. The change of variables formula in one dimension can be written

$$
p_X(x) = p_Z(z)􏰃\cdot\left| \frac{d}{dz}f(z) \right|^{-1},\qquad\text{(cf. equation \eqref{cov_f})}
$$

or equivalently as

$$
p_X(x) = p_Z(z)􏰃\cdot\left| \frac{d}{dx}f^{-1}(x) \right|.\qquad\text{(cf. equation \eqref{cov_finv})}
$$

_Sketch of proof._ For $f$ to be invertible, it must be strictly monotonic. That means that for all $x^{(1)}, x^{(2)}\in\mathbb{R}$ with $x^{(1)} < x^{(2)}$, we have $f(x^{(1)}) < f(x^{(2)})$ (strictly monotonically increasing) or $f(x^{(1)}) > f(x^{(2)})$ (strictly monotonically decreasing).

<img src="figures/change_of_variables_monotonic.pdf" alt="Monotonic functions" style="width: 600px;"/>
<center>Sketch of monotonic functions: (a) strictly increasing, (b) strictly decreasing</center>

Suppose first that $f$ is strictly increasing. Also let $F_X$ and $F_Z$ be the cumulative distribution functions of the random variables $X$ and $Z$ respectively. Then we have

$$
\begin{align}
F_X(x) &= P(X \le x)\\
&= P(f(Z) \le x)\\
&= P(Z \le f^{-1}(x))\qquad\text{(since $f$ is monotonically increasing)}\\
&= F_Z(f^{-1}(x))
\end{align}
$$

By differentiating on both sides with respect to $x$, we obtain the probability density function:

$$
\begin{align}
p_X(x) &= \frac{d}{dx}F_X(x)\\
&= \frac{d}{dx} F_Z(f^{-1}(x))\\
&= \frac{d}{dz}F_Z(z)\cdot\frac{d}{dx}f^{-1}(x)\\
&= p_Z(z)\frac{d}{dx}f^{-1}(x) \label{pdfx_inc}\tag{5}
\end{align}
$$

Now suppose first that $f$ is strictly decreasing. Then

$$
\begin{align}
F_X(x) &= P(X \le x)\\
&= P(f(Z) \le x)\\
&= P(Z \ge f^{-1}(x))\qquad\text{(since $f$ is monotonically decreasing)}\\
&= 1 - F_Z(f^{-1}(x))
\end{align}
$$

Again differentiating on both sides with respect to $x$:

$$
\begin{align}
p_X(x) &= \frac{d}{dx}F_X(x)\\
&= -\frac{d}{dx} F_Z(f^{-1}(x))\\
&= -F_Z'(f^{-1}(x))\frac{d}{dx}f^{-1}(x)\\
&= -p_Z(z)\frac{d}{dx}f^{-1}(x) \label{pdfx_dec}\tag{6}
\end{align}
$$

Now note that the inverse of a strictly monotonically increasing (resp. decreasing) function is again strictly monotonically increasing (resp. decreasing). This implies that the quantity $\frac{d}{dx} f^{-1}(x)$ is positive in \eqref{pdfx_inc} and negative in \eqref{pdfx_dec}, and so these two equations can be combined into the single equation:

$$
p_X(x) = p_Z(z)\left|\frac{d}{dx}f^{-1}(x)\right|
$$

which completes the proof.

#### Application to normalising flows
Normalising flows are a class of models that exploit the change of variables formula to estimate an unknown target data density. 

Suppose we have data samples $\mathcal{D}:=\{\mathbf{x}^{(1)}, \ldots, \mathbf{x}^{(n)}\}$, with each $\mathbf{x}^{(i)}\in\mathbb{R}^D$, and assume that these samples are generated i.i.d. from the underlying distribution $p_X$. 

A normalising flow models the distribution $p_X$ using a random variable $Z$ (also of dimension $D$) with a simple distribution $p_Z$ (e.g. an isotropic Gaussian), such that the random variable $X$ can be written as a change of variables $X = f_\theta(Z)$, where $\theta$ is a parameter vector that parameterises the smooth invertible function $f_\theta$. 

The function $f_\theta$ is modelled using a neural network with parameters $\theta$, which we want to learn from the data. An important point is that this neural network must be designed to be invertible, which is not the case in general with deep learning models. In practice, we often construct the neural network by composing multiple simpler blocks together. In TensorFlow Probability, these simpler blocks are the _bijectors_ that we will study in the first part of the week.

We use the principle of maximum likelihood to learn the optimal parameters $\theta$; that is:

$$
\begin{align}
\theta_{ML} &:= \arg \max_{\theta} P(\mathcal{D}; \theta)\\
&= \arg \max_{\theta} \log P(\mathcal{D}; \theta).
\end{align}
$$

In order to compute $\log P(\mathcal{D}; \theta)$ we can use the change of variables formula:

$$
\begin{align}
P(\mathcal{D}; \theta) &= \prod_{\mathbf{x}\in\mathcal{D}}  p_Z(f_\theta^{-1}(\mathbf{x})) \cdot\left|\hspace{0.1ex}\det J_{f_\theta^{-1}}(\mathbf{x}) \hspace{0.1ex}\right|\\
\log P(\mathcal{D}; \theta) &= \sum_{x\in\mathcal{D}} \log p_Z(f_\theta^{-1}(\mathbf{x})) + \log \hspace{0.1ex}\left|\hspace{0.1ex}\det J_{f_\theta^{-1}}(\mathbf{x}) \hspace{0.1ex}\right|\label{logliknf}\tag{7}
\end{align}
$$

The term $p_Z(f_\theta^{-1}(\mathbf{x}))$ can be computed for a given data point $\mathbf{x}\in\mathcal{D}$ since the neural network $f_\theta$ is designed to be invertible, and the distribution $p_Z$ is known. The term $\det J_{f_\theta^{-1}}(\mathbf{x})$ is also computable, although this also highlights another important aspect of normalising flow models: they should be designed such that the determinant of the Jacobian can be efficiently computed.

The log-likelihood \eqref{logliknf} is usually optimised as usual in minibatches, with gradient-based optimisation methods.

<a class="anchor" id="distributions"></a>
## Distributions

In this section we will look at `Distribution` objects in TensorFlow Probability, which are naturally one of the fundamental building blocks of the library. The main operations that we'll be using are sampling, and computing log-probabilities. 

A key point in understanding the interface and behaviour of these objects is that they are designed to perform vectorised computations for efficiency. This means that single objects are able to handle batches of distributions, samples, and log-probability computations. We'll see that this means we have to think quite carefully about the shapes of Tensors that we're using, and what these shapes mean. 

In [ ]:
import tensorflow as tf

We will use the following namespace for the TensorFlow Probability library.

In [ ]:
import tensorflow_probability as tfp
tfd = tfp.distributions

print(tfp.__version__)

#### Univariate distributions
We will first create some univariate distributions. There is a wide range of distributions available in the [distributions module](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions), of which we will only be using a few.

In [ ]:
# Create a univariate Normal distribution



In [ ]:
# Sample from the distribution



In [ ]:
# Draw multiple samples



In [ ]:
# We can pass a shape to the sample method



In [ ]:
# Plot some samples



In [ ]:
# Compute prob / log-prob of test points



In [ ]:
# Compute prob / log-prob of a batch of test points



A single `Distribution` object can represent a batch of distributions of the same type:

In [ ]:
# Create an exponential distribution



In [ ]:
# Create a batched exponential distribution



In [ ]:
# Sample from the distribution



We can see a first use of broadcasting when computing log-probabilities with a batched distribution.

In [ ]:
# Compute log-probs



#### Multivariate distributions
In the distributions seen so far, the `event_shape` property has been empty, indicating that the distribution is univariate. Here, we look at multivariate distributions.

In [ ]:
# Create a multivariate Gaussian distribution



In [ ]:
# Sample from the distribution



In [ ]:
# Plot samples from the multivariate Gaussian



In [ ]:
# Compute log-probs



We can also create a multivariate Gaussian using [`MultivariateNormalTriL`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalTriL), by passing in the lower triangular matrix $L$ such that $LL^T = \Sigma$, where $\Sigma$ is the covariance matrix. This is the Cholesky decomposition (see also [`tf.linalg.cholesky`](https://www.tensorflow.org/api_docs/python/tf/linalg/cholesky)).

In [ ]:
# Construct a multivariate Gaussian with MultivariateNormalTriL



In [ ]:
# Plot samples from the multivariate Gaussian



There are further ways of constructing a multivariate Gaussian: see the docs for [`MultivariateNormalDiagPlusLowRank`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiagPlusLowRank), [`MultivariateNormalFullCovariance`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalFullCovariance) and [`MultivariateNormalLinearOperator`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalLinearOperator).

Multivariate distributions can also be batched together, as in the following example.

In [ ]:
# Create a batched multivariate Gaussian



In [ ]:
# The batch and event shape are properties of the distribution



Of course we can also sample from a batched, multivariate distribution. The following shows the ordering of shapes that we should always keep in mind when working with `Distribution` objects:

`(sample_shape, batch_shape, event_shape)`

In [ ]:
# Sample from the batched multivariate Gaussian



_Exercise._ Take a look at the following Distribution object and call to the `log_prob` method. Work out what the shape of the resulting Tensor will be before you run the cell.

In [ ]:
mvn3 = tfd.MultivariateNormalDiag(loc=[[[2., 0., 0.5], [1., -0.5, 2.]]], scale_diag=[0.5, 1., 1.5])
test_pts = tf.random.normal((5, 1, 2, 1))
mvn3.log_prob(test_pts)

#### Independent distribution
The `Independent` distribution is often useful to manipulate batch and event shapes, and define multivariate distributions from univariate objects.

In [ ]:
# Create a batched Bernoulli distribution



In [ ]:
# Transfer the second batch dimension into the event space



In [ ]:
# Compute log_probs on both distributions



By default, the `Independent` distribution shifts all batch dimensions except the first into the event space. This can be changed with the `reinterpreted_batch_ndims` option:

In [ ]:
# Use the reinterpreted_batch_ndims keyword argument



In [ ]:
# Compute log-probs with the new distribution



_Exercise._ Construct a distribution object over a three-dimensional event space $(X_1, X_2, X_3)$, where each $X_i$ are independently distributed according to a Bernoulli distribution where the probability of a 0 event is equal to 0.9, 0.7 and 0.5 respectively. Use your distribution object to show that the log probability of the event $P(X_1, X_2, X_3) = (1, 1, 1)$ is equal to -4.199705.

<a class="anchor" id="bijectors"></a>
## Bijectors

In this section we will look at bijectors, which are another fundamental building block in TensorFlow Probability. Bijectors constitute the invertible and differentiable transformations that we will use to construct normalising flows. The [bijectors module](https://www.tensorflow.org/probability/api_docs/python/tfp/bijectors) has a range of in-built bijector functions, which can be composed to make complex transformations.

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

Two simple bijectors are the `Scale` and `Shift` bijectors.

In [ ]:
# Create Scale and Shift bijectors



In [ ]:
# Draw samples from a standard Normal distribution



In [ ]:
# Pass the samples through the forward method of each bijector



In [ ]:
# Plot the original and transformed samples

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(13, 4))

plt.subplot(1, 2, 1)
plt.hist(z.numpy(), bins=50)
plt.title("Original samples")

plt.subplot(1, 2, 2)
plt.hist(x.numpy(), bins=50)
plt.title("Transformed samples")
plt.show()

In [ ]:
# Chain the bijectors together



In [ ]:
# Pass the transformed samples through the inverse method of each bijector



#### Batched bijectors

In [ ]:
# Create a batched Softfloor bijector



In [ ]:
# Pass some test points through the bijector



In [ ]:
# Plot the transformed samples

plt.figure(figsize=(12, 5))
plt.plot(test_pts, transformed_pts[:, 0], label='temperature=0.01')
plt.plot(test_pts, transformed_pts[:, 1], label='temperature=0.1')
plt.plot(test_pts, transformed_pts[:, 2], label='temperature=1.')
plt.legend()
plt.show()

#### Computing log-probs of transformed samples

In [ ]:
# Create an Exp bijector



In [ ]:
# Compute the exp of the standard Normal samples



Recall the change of variables formula:

$$
\log p_X(\mathbf{x}) = \log p_Z(\mathbf{z}) - \log \hspace{0.1ex}\left|\det J_f(\mathbf{z}) \right|
$$

In [ ]:
# Use log_det_jacobian to compute log_probs of transformed samples



And using the inverse transformation:

$$
\log p_X(\mathbf{x}) = \log p_Z(\mathbf{z}) + \log \hspace{0.1ex}\left|\det J_{f^{-1}}(\mathbf{x}) \right|
$$

In [ ]:
# Repeat the calculation with the inverse_log_det_jacobian



#### The TransformedDistribution
The `TransformedDistribution` class provides a consistent API for distributions defined by bijectors and base distributions.

In [ ]:
# Define the log-normal distribution with TransformedDistribution



In [ ]:
# Confirm the log-probs of the transformed samples are the same as under a log-normal distribution



`TransformedDistribution` objects can also be defined by calling the bijector on the base distribution.

In [ ]:
# Recreate the log-normal TransformedDistribution



The `TransformedDistribution` infers the batch shape by broadcasting the batch shapes of the base distribution and the bijector.

In [ ]:
# Create a TransformedDistribution from a batched bijector



In [ ]:
# Test the new TransformedDistribution



In [ ]:
# Set a scaling lower triangular matrix



In [ ]:
# Define a bijector that operates on a rank >= 1 event space



In [ ]:
# Define TransformedDistribution with a batch and event shape



In [ ]:
# Sample from the transformed distribution



_Exercise._ Construct the distribution $\mathcal{N}(\mu, \Sigma)$, where $\mu = [0.5, -0.5]^T$ and $\Sigma = \left[\begin{array}{cc} 2 & 1\\ 1 & 2\end{array}\right]$, first using a `tfd.MultivariateNormalTriL` object, and then using a `tfd.TransformedDistribution` object with a zero-mean Gaussian with identity covariance matrix as a base distribution. Verify that the two representations are mathematically equivalent by computing log probs on a given sample.

<a class="anchor" id="nicerealnvp"></a>
## NICE / RealNVP

#### NICE
NICE stands for "nonlinear independent components estimation", and is a deep learning architecture framework for density estimation tasks. A key motivation for the proposed framework given in the abstract of the [original paper](#Dinh15) is as follows:

> It is based on the idea that a good representation is one in which the data has a distribution that is easy to model. For this purpose, a non-linear deterministic transformation of the data is learned that maps it to a latent space so as to make the transformed data conform to a factorized distribution, i.e., resulting in independent latent variables.

As with many normalising flow examples, a typical choice for a base distribution would be an isotropic Gaussian, which is then transformed by the deep learning model. An important aspect is the efficient calculation of the Jacobian determinant of the transformation. 

In this section, we will describe the NICE architecture, and the RealNVP architecture that is built upon it. We will follow the exposition of the original papers, and think of the forward transformation as acting on the data input example. Note however that this is in contrast to the usual bijector convention of using the forward transformation for sampling, and the inverse transformation for computing log probs.

#### Affine coupling layer
The basic building block of the NICE architecture is the affine coupling layer. Given an input $\mathbf{x}\in\mathbb{R}^D$, we split it into two blocks $(\mathbf{x}_{1:d}, \mathbf{x}_{d+1:D})$, where $d<D$ (usually $d\approx D / 2$), and apply a transformation of the form

$$
\begin{align}
\mathbf{z}_{1:d} &= \mathbf{x}_{1:d},\label{nice_acl1}\tag{1}\\
\mathbf{z}_{d+1:D} &= \mathbf{x}_{d+1:D} + t(\mathbf{x}_{1:d}),\label{nice_acl2}\tag{2}\\
\end{align}
$$

where $t:\mathbb{R}^d\mapsto\mathbb{R}^{D-d}$ is an arbitrarily complex function, such as a neural network. It is easy to see that the coupling layer as above has an identity Jacobian matrix, and is trivially invertible:

$$
\begin{align}
\mathbf{x}_{1:d} &= \mathbf{z}_{1:d},\\
\mathbf{x}_{d+1:D} &= \mathbf{z}_{d+1:D} - t(\mathbf{z}_{1:d}).\\
\end{align}
$$

Several coupling layers can be composed together to obtain a more complex, layered transformation. Note that a coupling layer leaves part of its input unchanged, and so the roles of the two subsets should be interchanged in alternating layers. 

If we examine the Jacobian, we can see that at least three coupling layers are needed to allow all dimensions to influence each other (this is left as an exercise for the reader). In the NICE paper, networks were composed with four coupling layers.

#### RealNVP
RealNVP stands for real-valued, non-volume preserving ([Dinh et al 2017](#Dinh17)). It was a follow-up work to the NICE paper, in which the affine coupling layer was modified as follows:

$$
\begin{align}
\mathbf{z}_{1:d} &= \mathbf{x}_{1:d},\label{realnvp_acl1}\tag{3}\\
\mathbf{z}_{d+1:D} &= \mathbf{x}_{d+1:D}\odot \exp(s(\mathbf{x}_{1:d})) + t(\mathbf{x}_{1:d}),\label{realnvp_acl2}\tag{4}\\
\end{align}
$$

where $s$ and $t$ stand for scale and translation, and are both functions that map from $\mathbb{R}^d$ to $\mathbb{R}^{D-d}$. The name RealNVP emphasises the fact that the transformation \eqref{realnvp_acl1}-\eqref{realnvp_acl2} is no longer volume-preserving, as is the case with \eqref{nice_acl1}-\eqref{nice_acl2}, due to the additional scaling provided by the term $\exp(s(\mathbf{x}_{1:d}))$. We use the network output $s(\mathbf{x}_{1:d})$ as a log-scale parameter for numerical stability.

As before, the inverse transformation is no more complex than the forward propagation:

$$
\begin{align}
\mathbf{x}_{1:d} &= \mathbf{z}_{1:d},\label{realnvp_inv_acl1}\tag{5}\\
\mathbf{x}_{d+1:D} &= (\mathbf{z}_{d+1:D} - t(\mathbf{z}_{1:d})) \odot \exp(-s(\mathbf{z}_{1:d})).\label{realnvp_inv_acl2}\tag{6}\\
\end{align}
$$

<img src="figures/affine_coupling_layer.png" alt="RealNVP: forward pass" style="width: 800px;"/>
<center>The forward and inverse passes of the RealNVP affine coupling layer</center>

Now, the Jacobian is given by

$$
\frac{\partial \mathbf{z}}{\partial \mathbf{x}} = \left[
\begin{array}{cc}
\mathbb{I}_d & \mathbf{0}\\
\frac{\partial \mathbf{z}_{d+1:D}}{\partial \mathbf{x}_{1:d}} & \text{diag}\,(\exp (s(\mathbf{x}_{1:d})))
\end{array}
\right]\in\mathbb{R}^{D\times D}
$$

and the log of the absolute value of the Jacobian determinant is easily calculated as $\sum_j s(\mathbf{x}_{1:d})_j$.

#### Spatial and channel-wise masking
Observe that the partitioning $\mathbf{x}\rightarrow (\mathbf{x}_{1:d}, \mathbf{x}_{d+1:D})$ can be implemented using a binary mask $b\in\{0, 1\}^{n_h\times n_w\times c}$, so that the forward pass \eqref{realnvp_acl1}-\eqref{realnvp_acl2} can be written

$$
\mathbf{z} = b\odot \mathbf{x} + (1-b)\odot(\mathbf{x}\odot \exp(s(b\odot \mathbf{x})) + t(b\odot\mathbf{x})).\label{realnvp_acl}\tag{7}
$$

Similarly, the inverse pass \eqref{realnvp_inv_acl1}-\eqref{realnvp_inv_acl2} can be written

$$
\mathbf{x} = b\odot \mathbf{z} + (1-b)\odot((\mathbf{z}-t(b\odot\mathbf{z}))\odot \exp(-s(b\odot \mathbf{z}))).\label{realnvp_inv_acl}\tag{8}
$$

RealNVP implements two types of masking for image data $\mathbf{x}\in\mathbb{R}^{n_h\times n_w\times c}$: spatial checkerboard and channel-wise masking. A spatial checkerboard mask applies the same partitioning to every channel dimension, as illustrated in the following figure.

<img src="figures/checkerboard_mask.png" alt="Checkerboard masking" style="width: 600px;"/>
<center>Spatial checkerboard masking in RealNVP. (a) A layer input $\mathbf{h}\in\mathbb{R}^{6\times 6\times 4}$ without masking, and (b) multiplied elementwise by a spatial checkerboard mask $b_s\in\{0, 1\}^{6\times 6}$, which is broadcast along the channel dimension</center>

A channel mask instead operates along the channel dimension, and applies the same partitioning at every spatial location, as in the following figure.

<img src="figures/channel_mask.png" alt="Channel masking" style="width: 600px;"/>
<center>Channel masking in RealNVP. (a) A layer input $\mathbf{h}\in\mathbb{R}^{6\times 6\times 4}$ without masking, and (b) multiplied elementwise by a channel mask $b_c\in\{0, 1\}^{4}$, which is broadcast across the spatial dimensions</center>

As in the NICE framework, we want to ensure that all dimensions are able to interact with each other. The RealNVP architecture consists of three layers of alternating checkerboard masks, where the partitions are permuted. 

<img src="figures/alternating_masks.png" alt="Alternating masks" style="width: 900px;"/>
<center>Three affine coupling layers, with alternating masks in between layers</center>

#### Squeeze operation
In the RealNVP architecture, after the three affine coupling layers with checkerboard masking there is a squeeze operation, where the spatial dimensions of the layer are divided into $2\times 2\times c$ subsquares, and reshaped into $1\times 1\times 4c$. The figure below illustrates this operation for a single channel:

<img src="figures/squeeze.png" alt="Squeeze operation" style="width: 600px;"/>
<center>The squeeze operation. The spatial dimensions are halved, and the channel dimension is quadrupled</center>

Following the squeeze operation, there are three more affine coupling layers, this time using channel masking, and again permuting the partitions between each layer.

#### Multiscale architecture
The final component of the RealNVP framework is the multiscale architecture. With the squeeze operation, the spatial dimensions are downsampled, but the channel dimensions are increased. In order to reduce the overall layer sizes in the deeper layers, dimensions are factored out as latent variables at regular intervals.

After one of the blocks of coupling-squeeze-coupling described above, half of the dimensions are factored out as latent variables, while the other half is further processed through subsequent layers. 

<img src="figures/factor_out_latent_variables.png" alt="Multiscale architecture" style="width: 800px;"/>
<center>Example showing how latent variables are factored out in the multiscale architecture. A layer input $\mathbf{h}^{(k)}\in\mathbb{R}^{8\times 8\times 2}$ will be reshaped to a $4\times4\times8$-shaped tensor after the coupling-squeeze-coupling block. Half of this tensor is absorbed into the base distribution as a latent variable $\mathbf{z}^{(k+1)}\in\mathbb{R}^{4\times 4\times 4}$ and the remainder $\mathbf{h}^{(k+1)}\in\mathbb{R}^{4\times 4\times 4}$ is processed through further layers of the network</center>

The complete RealNVP model has multiple levels of the multiscale architecture. This results in latent variables that represent different scales of features in the model. After a number of these levels, the final scale does not use the squeezing operation, and instead applies four affine coupling layers with alternating checkerboard masks to produce the final latent variable.

<img src="figures/realnvp.png" alt="Multiscale architecture" style="width: 800px;"/>
<center>The end-to-end RealNVP architecture. Each scale consists of a block of 3 coupling layers (with checkerboard mask), squeeze, 3 coupling layers (with channel mask), followed by half of the dimensions factored out as a latent variable. The final scale consists only of 4 coupling layers (with checkerboard mask) to produce the final latent variable</center>

The following summarises the forward pass $\mathbf{z} = f(\mathbf{x})$ of the overall architecture with $L$ scales. The functions $f^{(1)},\ldots,f^{(L-1)}$ consist of the coupling-squeeze-coupling block, whereas the function $f^{(L)}$ consists of 4 coupling layers with checkerboard masks.

>
>$$\begin{align}\mathbf{h}^{(0)}&=\mathbf{x}\\ (\mathbf{z}^{(k+1)}, \mathbf{h}^{(k+1)})&=f^{(k+1)}(\mathbf{h}^{(k)}),\qquad k=0,\ldots, L-2\\ \mathbf{z}^{(L)}&= f^{(L)}(\mathbf{h}^{(L-1)})\\
\mathbf{z} &= (\mathbf{z}^{(1)},\ldots,\mathbf{z}^{(L)})\end{align}$$
>

The latent variables factored out at each scale are reshaped and concatenated to produce a single latent variable $\mathbf{z} = (\mathbf{z}^{(1)},\ldots,\mathbf{z}^{(L)})$, which is assumed to be distributed according to a known base distribution (e.g. a diagonal Gaussian).

As a final note, the architecture described in this section was further developed with the Glow model ([Kingma and Dhariwal 2018](#Kingma18)), where the checkerboard and channel-wise masking was replaced with 1x1 convolutions.

<a class="anchor" id="bijector_subclassing"></a>
## Bijector subclassing

In this section we will build a partial implementation of the RealNVP architecture. In particular, we will use bijector subclassing to implement the affine coupling layer as a bijector object, using a binary mask:

$$
\begin{align}
\mathbf{z} &= b\odot \mathbf{x} + (1-b)\odot(\mathbf{x}\odot \exp(s(b\odot \mathbf{x})) + t(b\odot\mathbf{x})) & \text{(forward pass)}\\
\mathbf{x} &= b\odot \mathbf{z} + (1-b)\odot((\mathbf{z}-t(b\odot\mathbf{z}))\odot \exp(-s(b\odot \mathbf{z}))) & \text{(inverse pass)}
\end{align}
$$

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
# Create the AffineCouplingLayer class



We will build a `shift_and_log_scale_fn` with the following structure:

<img src="figures/shift_and_log_scale_fn.png" alt="Shift and log-scale network" style="width: 800px;"/>

In [ ]:
# Create an example shift_and_log_scale_fn



In [ ]:
# Define a binary mask



In [ ]:
# Test the AffineCouplingLayer



_Exercise._ Use the `forward_log_det_jacobian` and `inverse_log_det_jacobian` methods to compute the log Jacobian determinant on some dummy inputs. Verify that the `forward_log_det_jacobian` method gives the same as first computing the forward transformation, and then taking the negative of the `inverse_log_det_jacobian` method.

#### Two moons dataset
We will now create a normalising flow using the `AffineCouplingLayer` and train it on a two moons dataset.

In [ ]:
# Create the dataset



In [ ]:
# Visualise the dataset

import matplotlib.pyplot as plt

plt.scatter(train_data[:1000, 0], train_data[:1000, 1], alpha=0.2)
plt.title("Two moons data")
plt.show()

In [ ]:
# Create training and validation Datasets



#### Define and train the normalising flow

In [ ]:
# Define the bijectors chain



In [ ]:
# Define a base distribution



In [ ]:
# Define the transformed distribution



In [ ]:
# Define the model for training



In [ ]:
import os
from tensorflow.keras.callbacks import Callback

class SaveSamples(Callback):
    
    def __init__(self, plot_folder='./plots', **kwargs):
        super(SaveSamples, self).__init__(**kwargs)
        self.plot_folder = plot_folder
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch
        
    def plot(self, num):
        plt.figure(figsize=(8, 5))
        ax = plt.gca()
        plt.xlim([-1.5, 2.5])
        plt.ylim([-1, 1.5])
        ax.set_aspect('equal')
        samples = flow.sample(2000)
        plt.scatter(samples[:, 0], samples[:, 1], alpha=0.3)
        plt.tight_layout()
        plt.savefig("./plots/{:05d}.png".format(num))
        plt.close()
    
    def on_train_begin(self, logs=None):
        if not os.path.exists(self.plot_folder):
            os.makedirs(self.plot_folder)
        self.iteration = 0
        self.plot(self.iteration + 1)

    def on_train_batch_end(self, batch, logs=None):
        self.iteration += 1
        if self.iteration % 30 == 0:
            self.plot((self.iteration // 30) + 1)
        
save_samples = SaveSamples()

In [ ]:
# Compile and train the model



In [ ]:
# Plot the learning curves

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
plt.title("Loss vs epochs")
plt.ylabel("NLL")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
# Create a movie file

! ffmpeg -i ./plots/%05d.png -c:v libx264 -vf fps=10 -pix_fmt yuv420p -start_number 00000 samples.mp4

In [ ]:
from IPython.display import Video

Video("samples.mp4")

In [ ]:
# Plot the model transformations

samples = flow.sample(2000)
noise = realnvp_bijector.inverse(samples)

fig, axs = plt.subplots(3, 3, figsize=(15,10))
h = noise
for i, (bij, ax) in enumerate(zip(bijectors, axs.flat)):
    ax.scatter(h[:, 0], h[:, 1], alpha=0.1)
    ax.set_title(f"After {i} steps of flow")
    h = bij.forward(h)
axs[2, 2].scatter(samples[:, 0], samples[:, 1], alpha=0.1)
axs[2, 2].set_title("Model samples")
plt.show()

In [ ]:
# Clean up

filelist = [ f for f in os.listdir('./plots') if f.endswith(".png") ]
for f in filelist:
    os.remove(os.path.join('./plots', f))
if os.path.exists('samples.mp4'):
    os.remove('samples.mp4')

_Exercise._ Try re-running the two moons example again, but using a bi-modal base distribution. Is the flow able to more easily approximate the two moons distributions?

<a class="anchor" id="references"></a>
## References

<a class="anchor" id="Abdelhamed19"></a>
* Abdelhamed, A., Brubaker, M. A. & Brown, M. S. (2019), "Noise flow: Noise modeling with conditional normalizing flows", in *Proceedings of the IEEE International Conference on Computer Vision*, 3165–3173.
<a class="anchor" id="Dinh15"></a>
* Dinh, L., Krueger, D. & Bengio, Y. (2015),"NICE: Non-linear Independent Components Estimation", in *3rd International Conference on Learning Representations, (ICLR)*, San Diego, CA, USA, May 7-9, 2015.
<a class="anchor" id="Dinh17"></a>
* Dinh, L., Sohl-Dickstein, J. & Bengio, S. (2017), "Density estimation using Real NVP",  in *5th International Conference on Learning Representations, (ICLR)*, Toulon, France, April 24-26, 2017.
<a class="anchor" id="Ho19"></a>
* Ho, J., Chen, X., Srinivas, A., Duan, Y., & Abbeel, P. (2019), "Flow++: Improving flow-based generative models with variational dequantization and architecture design", in *Proceedings of the 36th International Conference on Machine Learning, ICML*.
<a class="anchor" id="Kingma18"></a>
* Kingma, D. P. & Dhariwal, P. (2018), "Glow: Generative Flow with Invertible 1x1 Convolutions", in *Advances in Neural Information Processing Systems*, **31**, 10215--10224.
<a class="anchor" id="Kumar19"></a>
* Kumar, M., Babaeizadeh, M., Erhan, D., Finn, C., Levine, S., Dinh, L. & Kingma, D. (2019), "VideoFlow: A Flow-Based Generative Model for Video", in *Workshop on Invertible Neural Nets and Normalizing Flows*, ICML, 2019.
<a class="anchor" id="Larochelle11"></a>
* Larochelle, H. & Murray, I. (2011), "The Neural Autoregressive Distribution Estimator", *Proceedings of Machine Learning Research*, **15**, 29-37.
<a class="anchor" id="Prenger19"></a>
* Prenger, R., Valle, R., & Catanzaro, B. (2019), "Waveglow: A flow-based generative network for speech synthesis", in *Proceedings of the International Conference on Acoustics, Speech, and Signal Processing (ICASSP)*, IEEE, 3617-3621.
<a class="anchor" id="Rezende15"></a>
* Rezende, D. & Mohamed, S. (2015), "Variational Inference with Normalizing Flows", in *Proceedings of Machine Learning Research*, **37**, 1530-1538.
<a class="anchor" id="vandenOord16a"></a>
* van den Oord, A., Kalchbrenner, N. & Kavukcuoglu, K. (2016a), "Pixel Recurrent Neural Networks", *Proceedings of Machine Learning Research*, **48**, 1747-1756.
<a class="anchor" id="vandenOord16b"></a>
* van den Oord, A., Kalchbrenner, N., Espeholt, L., Kavukcuoglu, K., Vinyals, O. & Graves, A. (2016b), "Conditional Image Generation with PixelCNN Decoders", *Advances in Neural Information Processing Systems*, **29**, 4790-4798.
<a class="anchor" id="vandenOord16c"></a>
* van den Oord, A., Dieleman, S., Zen, H., Simonyan, K., Vinyals, O., Graves, A., Kalchbrenner, N., Senior, A. & Kavukcuoglu, K. (2016c), "WaveNet: A Generative Model for Raw Audio", arXiv preprint, abs/1609.03499.
<a class="anchor" id="vandenOord18"></a>
* van den Oord, A., Li, Y., Babuschkin, I., Simonyan, K., Vinyals, O., Kavukcuoglu, K., van den Driessche, G., Lockhart, E., Cobo, L., Stimberg, F., Casagrande, N., Grewe, D., Noury, S., Dieleman, S., Elsen, E., Kalchbrenner, N., Zen, H., Graves, A., King, H., Walters, T., Belov, D., & Hassabis, D. (2018), "Parallel WaveNet: Fast high-fidelity speech synthesis", in *Proceedings of the 35th International Conference on Machine Learning*, **80**, 3918–3926.